<a href="https://colab.research.google.com/github/osullik/ic23050/blob/main/EnrichedData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
from google.colab import drive
import json
import pandas as pd
from collections import defaultdict

In [115]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Load in the Data Files

In [116]:

dataDictFile =  "/content/gdrive/MyDrive/InfoChallenge/ic23050/L4 Washington Fatal Crash Files - Washington Traffic Safety Commission/WTSC_DataDictionary_flattened.json"
dataFile =      "/content/gdrive/MyDrive/InfoChallenge/ic23050/L4 Washington Fatal Crash Files - Washington Traffic Safety Commission/Data_Washington_Fatal_Crash_Survey_with_zip.csv"

with open(dataDictFile) as f:
  dataDict = json.load(f)

crashDF = pd.read_csv(dataFile)


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,23,24,302,304) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
crashDF

,Unnamed: 0,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,holiday,...,lab,isedtcase,crf1,crf2,crf3,CoRoadName,CoMP,IntCoRoadName,IntCoMP,zipcode
0,0,2017,1,E628946,2.0,01/01/2017,2:12,1,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98201.0
1,1,2017,2,E627989,26.0,01/02/2017,17:14,2,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98359.0
2,2,2017,4,3747633,263.0,01/01/2017,18:47,1,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98103.0
3,3,2017,5,E628691,4.0,01/01/2017,3:50,1,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99219.0
4,4,2017,6,3746306,263.0,01/05/2017,9:53,5,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98174.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4127,4127,2021,636,Incid08,334.0,08/08/2021,9:38,8,8,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,99155.0
4128,4128,2021,638,Incid09,334.0,09/05/2021,1:36,5,9,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3971.0
4129,4129,2021,639,EB48605,263.0,04/21/2021,17:32,21,4,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,98144.0
4130,4130,2021,639,EB48605,263.0,04/21/2021,17:32,21,4,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,98144.0


In [16]:
#This code takes a deep data dictionary of the WTSC data and flattens it
flattenedDict = defaultdict(dict)

#Flatten the keys from the "Crash_Data"
for key in dataDict["Crash_Data"].keys():
  if len(dataDict["Crash_Data"][key]["DATA"].keys()) != 0:                      #If there are no dict entries, is continuous variable
    for subkey in dataDict["Crash_Data"][key]["DATA"].keys():                   #Iterate through each key
      flattenedDict[key][subkey] = dataDict["Crash_Data"][key]["DATA"][subkey]  #Add to the flattened dictionary
  else:
    flattenedDict[key] = None

#Flatten the keys from the "Person_Data", where keys (table values) overlap
  #they will just update the dictionary with any other subvalues (should be consistent)
for key in dataDict["Person_Data"].keys():
  if len(dataDict["Person_Data"][key]["DATA"].keys()) != 0:
    for subkey in dataDict["Person_Data"][key]["DATA"].keys():
      flattenedDict[key][subkey] = dataDict["Person_Data"][key]["DATA"][subkey]
  else:
    flattenedDict[key] = None


KeyError: ignored

In [17]:
#Output the flattened Dictionary to the File
with open('/content/drive/MyDrive/InfoChallenge/ic23050/L4 Washington Fatal Crash Files - Washington Traffic Safety Commission/WTSC_DataDictionary_flattened.json', 'w') as g:
    json.dump(flattenedDict, g, indent=4)

OSError: ignored

Note: a moderate number of manual revisions were made after flattening to correct for errors discovered in parsing. The dictionary imported below is not the same one exported here for this reason

# Modify the Spreadsheet


In [129]:
#Import a nested dictionary of the form {columnName:{Key,Value}}
flatDictFile = "/content/gdrive/MyDrive/InfoChallenge/ic23050/L4 Washington Fatal Crash Files - Washington Traffic Safety Commission/WTSC_DataDictionary_flattened.json"
with open(flatDictFile) as f:
  flatDict = json.load(f)

crashDF = pd.read_csv(dataFile)

In [139]:
#Take the encoded data and use the data dictionary to transform it into 
#a human-readable format for analysis. 

#It steps through each column, then each row, looking up the encoded value in
#a data dictionary and recording the human-readable value. Those values are
#Collected in a list, and then added as the values of a dictionary, with the 
#key being the column name. That dictionary is transformed into a dataframe
#and the dataframe output to CSV

verboseDict = {}                                                                #Dict to hold the output
verboseFrame = pd.DataFrame                                                     #Dataframe to hold the output

for column in crashDF.columns:                                                  #For each column (variable)
                                                                                
  missingKeys = set(())                                                         #Used in error checking
  leadZeroKeys = set(())                                            
  valueList = []                                                                #will collect the transformed vals

  if column in flatDict.keys():                                                 #Condition column is in data dict (several are not)
    
    if flatDict[column] == "null" or flatDict[column] == None:                  #Condition that the column isnt void
      
      for idx, series in crashDF.iterrows():                                    #Add the continuous variables to the list
        val = series[column]
        valueList.append(val)
    
    else:                                                                       #Condition column IS in the data dict

      for idx, series in crashDF.iterrows():                                    #Step through each row
        val = series[column]                                                    #val === the current cell in DF
        
        if val == None or val == "null" or str(val) == "nan":                   #Add null values as they are to protect type casting below 
          valueList.append(val)
          
        else:                                                                   #If not null

          if dtypeDict[column] == "float64":                                    #Identify floats and remove decimal points so they keys will match data dict
            
            try:
              val = str(int(val))                                               #Transform the float to an int, then string to allow matcing with data dict
              valueList.append(dataDict[column][val])
            
            except KeyError:                                                    #If it's not in the data dict
              try:
                leadZeroVal = "0"+val                                           #Most likely missing a leading zero
                valueList.append(dataDict[column][leadZeroVal])                  #Add leading zero and carry on
                leadZeroKeys.add(leadZeroVal)
              except KeyError:                                                  #If still not in dict, could be missing second leading zero
                try:                                                              #Add second leading zero and try again
                  leadZeroVal = "00"+val
                  valueList.append(dataDict[column][leadZeroVal])
                  leadZeroKeys.add(leadZeroVal)
                except KeyError:                                                #If still not in dict, likely missing entry
                  missingKeys.add(val)                                            #add as it is
                  valueList.append(val)
                  pass
         
          else:                                                                 #If not a float, basically the same process
            try:                                                                  #as above except no int typecasting
              val = str(val)
              valueList.append(dataDict[column][val])
            except KeyError:
              try:
                leadZeroVal = "0"+val
                valueList.append(dataDict[column][leadZeroVal])
                leadZeroKeys.add(leadZeroVal)
              except KeyError:
                try:
                  leadZeroVal = "00"+val
                  valueList.append(dataDict[column][leadZeroVal])
                  leadZeroKeys.add(leadZeroVal)
                except KeyError:
                  missingKeys.add(val)
                  valueList.append(val)
                  pass
      
      #if len(missingKeys) != 0:                                                #Debug: identify missing keys
        #print(column, "Missing Keys", missingKeys)
      #if len(leadZeroKeys) != 0:
        #print(column, "needed Leading Zeros", leadZeroKeys)

  
            
  else:

    for idx, series in crashDF.iterrows():                                      #If the column not in data dict at all
      val = series[column]                                                        #Add values as they are
      valueList.append(val)

      

  verboseDict[column] = valueList
  #print("added:", column)

verboseFrame = pd.DataFrame.from_dict(verboseDict)                              #Convert the dict into a data frame
  
    

added: Unnamed: 0
added: year
added: case
added: par
added: repjur
added: crash_dt
added: crash_tm
added: accday
added: accmon
added: holiday
added: county
added: co_char
added: city
added: weather
added: weather2
added: lightcond
added: surfcond
added: surftype
added: numfatal
added: nmforms
added: pforms
added: vforms
added: not_tm
added: arr_tm
added: hosp_tm
added: fhevent
added: hitrun
added: manncol
added: milepnt
added: reljunc
added: reljuncinter
added: intersectiontype
added: funcsystem
added: landuse
added: ownership
added: route
added: schlbus
added: spejur
added: contdev
added: devfunc
added: trafid1
added: trafid2
added: x
added: y
added: wk_zone
added: vnumber
added: crashtype
added: criticalevent
added: criticaleventcat
added: spdlim
added: body
added: trailer
added: emerg
added: hazcargo
added: leave
added: mhevent
added: roadalgn
added: roadprof
added: traflane
added: trafflow
added: vehmanvr
added: numoccs
added: regowner
added: regstate
added: spuse
added: make
added

In [141]:
#Debug Code to identify missing columns in the dataset (all accounted for)
  #Rows and columns accounted for by count in the Dataframe Dimensions.
ocols = []
ncols = []
for col in crashDF.columns:
  ocols.append(col)
for col in verboseFrame.columns:
  ncols.append(col)

print(list(set(ocols)-set(ncols)))

[]


Missing Keys in the Data Dict, recorded for posterity


```
repjur Missing Keys {'349'}
co_char Missing Keys {'Pend Oreille'}
city Missing Keys {'171'}
surfcond Missing Keys {'10', '0', '98', '11', '99'}
surftype Missing Keys {'0'}
reljunc Missing Keys {'20'}
contdev Missing Keys {'65', '97'}
devfunc Missing Keys {'8'}
criticalevent Missing Keys {'21', '20'}
leave Missing Keys {'7'}
mhevent Missing Keys {'59'}
roadprof Missing Keys {'5', '6', '8', '0'}
regstate Missing Keys {'0'}
model Missing Keys {'10', '804', '907', '999', '808', '998', '809', '997', '806'}
seqevent2 Missing Keys {'59'}
seqevent3 Missing Keys {'59'}
seqevent4 Missing Keys {'59'}
seqevent5 Missing Keys {'59'}
seqevent6 Missing Keys {'59'}
restraint Missing Keys {'29'}
commlic Missing Keys {'7'}
dricond1 Missing Keys {'0'}
alctst Missing Keys {'10'}
agegrp1 Missing Keys {'18-20', '56-60', '26-30', '66-70', '46-50', '81+', '41-45', '71-75', '31-35', '51-55', '16-17', '10-15', '76-80', '21-25', '36-40', '61-65'}
agegrp2 Missing Keys {'26-35', '46-55', '76+', '0-15', '16-25', '66-75', '36-45', '56-65'}
roadclass Missing Keys {'State route', 'US highway', 'County road', 'Other', 'City street', 'Interstate', 'Unknown'}
dayweek Missing Keys {'Wednesday', 'Tuesday', 'Sunday', 'Monday', 'Friday', 'Saturday', 'Thursday'}
restuse Missing Keys {'Belt Used', 'Not Used', 'Unknown'}
vehtype Missing Keys {'NR-U'}
thc_res Missing Keys {'560'}
cthc_res Missing Keys {'850'}
lab Missing Keys {'wa nm'}
```



In [143]:
#Output the transfored data to the CSV
verboseFrame.to_csv("/content/gdrive/MyDrive/InfoChallenge/ic23050/L4 Washington Fatal Crash Files - Washington Traffic Safety Commission/verboseData.csv")